In [1]:
# Load our libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Use a nicer style for plots
plt.style.use("seaborn-v0_8-muted")

# Import the regression tree from scikit-learn and a plotting helper
from sklearn.tree import DecisionTreeRegressor, plot_tree
# Import our train_test_split helper
from sklearn.model_selection import train_test_split

In [9]:
dataset = pd.read_csv("/Users/rubenstark/Documents/GitHub/Its-Wekk/4 - Data/4 - Johannes/merged_df.csv")


Prepare Dataset into target and features and split them into test train 

In [ ]:
# Split our data intro features and targets
# Teile das Dataset in Features und Zielvariable
y = dataset["target_column"]  # Zielvariable
X = dataset.drop(columns=["target_column"])  # Alle Spalten außer der Zielvariable

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=72)

# Build a Tree and fit it to the train Data



## Aufbau des Entscheidungsbaums
1. **Feature-Space-Aufteilung**: Der gesamte Merkmalsraum \( X \) wird in \( J \) Regionen \( R_1, R_2, \dots, R_J \) unterteilt.
2. **Vorhersage in den Regionen**: Für jede Region wird der Durchschnitt der Zielvariablen in dieser Region als Vorhersage verwendet.

Beispiel: Ein Entscheidungsbaum mit zwei Regionen \( R_1 \) und \( R_2 \) könnte Vorhersagen von 5 und 12 liefern, je nachdem, in welche Region ein neues Beispiel fällt.

---

## Aufteilung des Merkmalsraums
- **Vorgehen**: Der Merkmalsraum wird mit einem rekursiven, "top-down"-Ansatz (greedy) aufgeteilt:
  - **Schritt 1**: Für jedes Merkmal \( X_k \) werden mögliche Splitpunkte \( s \) getestet, die den Raum in zwei Regionen teilen:
    \[
    R_1 = \{x \mid x_k \leq s\}, \quad R_2 = \{x \mid x_k > s\}
    \]
  - **Schritt 2**: Die Aufteilung wird basierend auf einer Loss-Funktion bewertet (z. B. Mean Squared Error).
  - **Schritt 3**: Der Splitpunkt \( s \) und das Merkmal \( X_k \), die die Loss-Funktion minimieren, werden ausgewählt.

- **Rekursion**: Dieser Prozess wird wiederholt, indem jede Region weiter unterteilt wird, bis ein Abbruchkriterium erreicht wird (z. B. maximale Baumtiefe oder minimale Anzahl an Datenpunkten in einer Region).

---

## Ziel
Der Baum wird so aufgebaut, dass er die Zielvariable \( Y \) in den Regionen optimal vorhersagt, indem er die Varianz minimiert und die Struktur des zugrunde liegenden Datensatzes berücksichtigt.

In [ ]:
# DecisionTreeRegressor is just another sklearn estimator, we can use it
# like we would any other model
Decision_tree = DecisionTreeRegressor()
Decision_tree.fit(X_train, y_train) #Habebn wir das jetzt irgendwo festgehalten/definiert? 

In [ ]:
Decision_tree.tree_.max_depth

In [ ]:
fig, ax = plt.subplots(figsize=(16, 10))
plot_tree(Decision_tree, label="root", filled=True, max_depth=1, 
          feature_names=features, ax=ax);

Use ccp_alpha (and other parameters) to optimize the Decision Tree for example when it comes to overfitting

[`DecisionTreeRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)

In [ ]:
criterion='squared_error', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, mintree = DecisionTreeRegressor(_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, ccp_alpha=0.0, monotonic_cst=None)
tree.fit(X_train, y_train)

Crossvalidation to find best alpha

In [3]:
# Import our tools for model selection
from sklearn.model_selection import cross_validate, KFold

In [ ]:
# Specify our CV strategy
cv = KFold(n_splits=5, shuffle=True, random_state=72)

# Specify the values of cost-complexity parameter to try
alphas = np.logspace(-4, 0, 200) #erzeugt werte von 10^-4 bis 10^0)

scores = [] # Store the cross-validation scores
scores_std = [] # Store the standard deviations of the scores

# Perform cross-validation
for alpha in alphas:
    tree_cv = DecisionTreeRegressor(ccp_alpha=alpha)
    # Cross validate the tree, 
    cv_results = cross_validate(tree_cv, X_train, y_train, cv=cv, 
                                scoring="neg_mean_squared_error")
    scores.append(-cv_results["test_score"].mean())
    scores_std.append(cv_results["test_score"].std())

Bagging

Bagging (Bootstrap Aggregating) ist eine Technik, um die Stabilität und Genauigkeit von Machine-Learning-Algorithmen zu verbessern, insbesondere bei Modellen wie Entscheidungsbäumen, die anfällig für hohe Varianz sind. Es basiert auf dem Bootstrapping-Prinzip, bei dem mehrere Trainingssets durch Zufallsstichproben mit Zurücklegen erzeugt werden.

Jeder Baum wird auf einem dieser zufälligen Datensets trainiert, und die Vorhersagen der B Modelle werden durch Mittelung kombiniert. Mathematisch reduziert Bagging die Varianz der Modelle, weil unabhängige Fehler über die Modelle hinweg geglättet werden. So wird die Vorhersage insgesamt stabiler und robuster gegen Variationen in den Trainingsdaten.

Das Ziel ist, Vorhersagefehler durch Mittelung der Outputs der individuellen Modelle zu minimieren, was insgesamt zu einer besseren Modellleistung führt.

In [ ]:
# Import the sklearn implementation of bagging
from sklearn.ensemble import BaggingRegressor

# Create a bagged tree estimator with B=100 trees
bagged_trees = BaggingRegressor(DecisionTreeRegressor(), n_estimators=B)

# Fit the bagged estimator and compute the MSE on the training set
bagged_trees.fit(X_train, y_train)

# Compute the predictions on the training and test sets
y_pred_train_bag = bagged_trees.predict(X_train)
y_pred_test_bag = bagged_trees.predict(X_test)

print("Train MSE: ", mse(y_train, y_pred_train_bag))
print("Test MSE : ", mse(y_test, y_pred_test_bag))

In [ ]:
for i in range(n_bootstrap):
    indices = np.random.choice(np.arange(n_samples), size=n_samples, replace=True)
    X_bootstrap = X2[indices]
    Y_bootstrap = Y[indices]
    predicted_sales[i] = make_prediction(X_bootstrap, Y_bootstrap, 20000)

Random Forrest

Random Forests erweitern Bagging, indem sie jedem Baum eine zusätzliche Zufallskomponente hinzufügen. Jeder Baum wird mit einem bootstrap-Sample der Trainingsdaten trainiert, wobei nur ein zufälliger Teil der Features für die Konstruktion des Baums verwendet wird. Dadurch unterscheidet sich Random Forests von klassischem Bagging, bei dem alle Features verfügbar sind.

Die zufällige Auswahl der Features reduziert die Korrelation zwischen den Bäumen und verbessert die Generalisierung des Modells. Üblicherweise wird die Anzahl der verwendeten Features  m  so gewählt, dass  m \approx \sqrt{p} , wobei  p  die Gesamtzahl der Features ist. Wenn  m = p  gesetzt wird, ist Random Forest gleichbedeutend mit einem Bagging-Modell.

In [ ]:
# Import the random forest regressor
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=B, max_features="sqrt")
rf.fit(X_train, y_train)

# Compute the predictions on the training and test sets
y_pred_train_rf = rf.predict(X_train)
y_pred_test_rf = rf.predict(X_test)

print("Train MSE: ", mse(y_train, y_pred_train_rf))
print("Test MSE : ", mse(y_test, y_pred_test_rf))

Der Unterschied zwischen der Verwendung eines Integers oder eines Floats bei der Angabe von max_features in einem Random Forest Modell (wie in Scikit-learn) liegt in der Bedeutung des Parameters und wie die Anzahl der maximal zu betrachtenden Features berechnet wird:

1. Wenn max_features ein Integer ist:

	•	Der Wert gibt die exakte Anzahl der maximal zu betrachtenden Features an, die bei der Teilung eines Knotens in jedem Decision Tree berücksichtigt werden sollen.
	•	Beispiel: max_features=3 bedeutet, dass 3 Features aus dem gesamten Feature-Set zufällig ausgewählt werden, um die beste Teilung zu bestimmen.

2. Wenn max_features ein Float ist:

	•	Der Wert gibt einen Prozentsatz der verfügbaren Features an, die verwendet werden sollen. Der Float-Wert muss zwischen 0.0 und 1.0 liegen.
	•	Beispiel: max_features=0.5 bedeutet, dass 50 % der Features (aufgerundet) zufällig ausgewählt werden, um die beste Teilung zu bestimmen.
    

In [ ]:
target_data = pd.read_csv("/Users/rubenstark/Documents/GitHub/Its-Wekk/4 - Data/Final_Data/Cleaned/10Targcleaned.csv")

In [ ]:
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose

# Sicherstellen, dass 'Datum' ein datetime-Format hat
target_data['Datum'] = pd.to_datetime(target_data['Datum'])
target_data.set_index('Datum', inplace=True)

# Auswahl der Zeitreihe (PM10 Mittelwerte)
series = target_data['PM10 (Stundenmittelwerte)']

# 1. Globalen Trend bestimmen und entfernen
result_trend = seasonal_decompose(series, model='additive', period=len(series))  # Gesamter Trend
trend = result_trend.trend
series_detrended = series - trend  # Serie ohne Trend

# 2. Wöchentliche Saisonalität entfernen
result_weekly = seasonal_decompose(series_detrended, model='additive', period=168)  # Wochen-Saisonalität
weekly_seasonal = result_weekly.seasonal
series_no_weekly = series_detrended - weekly_seasonal  # Serie ohne wöchentliche Saisonalität

# 3. Tägliche Saisonalität entfernen
result_daily = seasonal_decompose(series_no_weekly, model='additive', period=24)  # Tages-Saisonalität
daily_seasonal = result_daily.seasonal
series_residual = series_no_weekly - daily_seasonal  # Serie ohne tägliche Saisonalität

# Neues DataFrame mit den Residuen (ohne Trend, Wochen- und Tages-Saisonalität)
residual_data = series_residual.reset_index()
residual_data.columns = ['Datum', 'PM10_Residual']

# NaN-Werte entfernen (falls Trend/Saisonalität die ersten/letzten Werte beeinflussen)
residual_data.dropna(inplace=True)

# Residuen-Dataset anzeigen
print(residual_data.head())

# Optional: Als CSV speichern
#residual_data.to_csv("PM10_Residuals_NoTrend_NoSeasonality.csv", index=False)